In [3]:
n_exp = 10

algorithms  [ "OzaBag","HoeffdingAdaptiveTree"]#, "RebalanceStream","CSMOTE",
#     #"new" : ["improved_OOB","improved_UOB","OOB","UOB","WEOB1","WEOB2","RebalanceStream"] 


datasets = {
    "real" : ["Elec","PAKDD", "KDDCup"],
    "art1" : ["sea_", "sine_","seaG_","sineG_"],
    "art2" : ["incremental","sudden", "periodic"]
}

directories = {
    "art1" : ["P(y)","P(x_y)","P(y_x)"],
    "res_art1" : ["P\\(y\\)","P\\(x_y\\)","P\\(y_x\\)"],
    "name_art1" : ["Py","Px_y","Py_x"],
    
    

    "art2" : ["appearing-minority","disappearing-minority","jitter","clusters-movement","appearing-clusters",
              "splitting-clusters", "borderline", "shapeshift", "minority-share"]
}




cnt = 0
options = [1,2,3,4]
test = open("Testtdock.sh","a")
i = 1
for i in range(1,n_exp+1): #experiment
    test.write(f"date\n")
    for o in options: #imbalance
        for di,d in enumerate(directories["art1"]):  #drift
            test.write(f"echo {i} {o} {alg} {directories['res_art1'][di]}\n")
            for data in datasets["art1"]: # SEA/SINE
                for alg in algorithms #algorithm
                
                    test.write(f"mkdir results/{directories['res_art1'][di]}/{data}{str(o)}_{str(10-o)}/{alg}\n")

                    if "G" in data:
                        drift = "45000 -j 10000"
                    else:
                        drift = "50000"



                    l = f'(meta.{alg})
                    if "Tree" in alg:
                        l = f'(trees.{alg})'
                    elif "ESOS" in alg:
                        l = f'(meta.{alg} -c (OS_ELM -p) -e (WELM -p))'
                    elif "Impr" in alg:
                        alg1 = ''.join(alg)
                        alg2 = alg.replace('_', '').replace("i","I")
                        alg = alg1
                        l = f'(meta.{alg2})'
                    elif "Reb" in alg:
                        l = f'(meta.{alg} -l trees.HoeffdingAdaptiveTree)'
                    elif "CS" in alg:
                        l = f'(meta.{alg} -l trees.HoeffdingAdaptiveTree)'


                    cnt += 1
                    test.write(f'docker run --rm --name="{directories["name_art1"][di]}_{alg}_{data}{str(o)}_{str(10-o)}_{str(i)}" '
                               f'-v $(pwd)/results:/src/results test_moa bash -c '
                               f'"java -Xmx14g -Xss50M -cp moa.jar -javaagent:sizeofag-1.0.4.jar moa.DoTask \\"'
                          f'EvaluatePrequential -l {l} -s '
                          f'(ArffFileStream -f ../datasets/SeaSine/{d}/{data}{str(o)},{str(10-o)}.arff) -e '
                          f'(WindowFixedClassificationPerformanceEvaluator -w {drift} -r -f -g) -i -1 -f 5000\\" '
                          f'1> results/{directories["res_art1"][di]}/{data}{str(o)}_{str(10-o)}/{alg}/{str(i)}_err.csv 2> results/{directories["res_art1"][di]}/{data}{str(o)}_{str(10-o)}/{alg}/{str(i)}.csv"\n') 

        for di,d in enumerate(directories["art2"]): #drift
            test.write(f"echo {i} {o} {alg} {d}\n")

            test.write(f"mkdir results/Drifts/{d}/{str(o)}/{alg}\n")
            for data in datasets["art2"]:  #speed
                for alg in algorithms #algorithm
                    if "sudden" in data:
                        drift = "50000"
                    else:
                        drift = "45000 -j 10000"


                    l = f'(meta.{alg})
                    if "Tree" in alg:
                        l = f'(trees.{alg})'
                    elif "ESOS" in alg:
                        l = f'(meta.{alg} -c (OS_ELM -p) -e (WELM -p))'
                    elif "Impr" in alg:
                        alg1 = ''.join(alg)
                        alg2 = alg.replace('_', '').replace("i","I")
                        alg = alg1
                        l = f'(meta.{alg2})'
                    elif "Reb" in alg:
                        l = f'(meta.{alg} -l trees.HoeffdingAdaptiveTree)'
                    elif "CS" in alg:
                        l = f'(meta.{alg} -l trees.HoeffdingAdaptiveTree)'



                    cnt += 1
                    test.write(f"java -Xmx14g -Xss50M -cp moa.jar -javaagent:sizeofag-1.0.4.jar moa.DoTask '"
                          f'EvaluatePrequential -l {l} -s '
                          f'(ArffFileStream -f ./drifts_arff/{d}-{data}-{o}-{i-1}.arff) -e '
                          f"(WindowFixedClassificationPerformanceEvaluator -w {drift} -r -f -g) -i 100000 -t 500 -f 5000' "
                               f"1> results/Drifts/{d}/{str(o)}/{alg}/{data}_{str(i)}_err.csv 2> results/Drifts/{d}/{str(o)}/{alg}/{data}_{str(i)}.csv\n")


test.close()
print(cnt)

480
